<a href="https://colab.research.google.com/github/NET-OF-BEING/API/blob/master/4k_Video_Upscaler_Colab_(Real_ESRGAN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4k Video Upscaler Colab (Real-ESRGAN)

Adapted from: [Real-ESRGAN](https://github.com/xinntao/Real-ESRGAN)

Made with ❤️ by: [yuvraj108c](https://github.com/yuvraj108c)

Github repository: https://github.com/yuvraj108c/4k-video-upscaler-colab

# 1. Setup (~1 minute)

In [7]:
import torch
assert torch.cuda.is_available(), "GPU not detected.. Please change runtime to GPU"

from PIL import Image
import cv2, os, subprocess
from google.colab import drive
from tqdm import tqdm

!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN

!pip install -q torch==2.0.1 torchvision==0.15.2 --extra-index-url https://download.pytorch.org/whl/cu118
!pip install -q basicsr facexlib gfpgan ffmpeg ffmpeg-python
!pip install torch==2.6.0
!pip install -q -r requirements.txt
!python setup.py develop

mount_drive=True #@param{type:"boolean"}

if mount_drive:
  drive.mount('/content/gdrive/')

SyntaxError: invalid syntax (<ipython-input-7-2f31dc2488bf>, line 14)

# 2. Upscale video

- The upscaled video will be saved to `output_dir`
- If google drive is mounted, it will be also saved at `MyDrive/Upscaled Videos (REAL-ESRGAN)`


In [5]:
video_path="/content/Real-ESRGAN/DerekJess.mp4" #@param{type:"string"}
output_dir="/content/" #@param{type:"string"}
resolution = "4k (3840 x 2160)" # @param ["FHD (1920 x 1080)", "2k (2560 x 1440)", "4k (3840 x 2160)","2 x original", "3 x original", "4 x original"] {type:"string"}
model = "RealESRGAN_x4plus" #@param ["RealESRGAN_x4plus" , "RealESRGAN_x4plus_anime_6B", "realesr-animevideov3"]

assert os.path.exists(video_path), "Video file does not exist"

video_capture = cv2.VideoCapture(video_path)
video_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
video_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

final_width = None
final_height = None
aspect_ratio = float(video_width/video_height)

# Get output resolutions
match resolution:
  case "FHD (1920 x 1080)":
    final_width=1920
    final_height=1080
  case "2k (2560 x 1440)":
    final_width=2560
    final_height=1440
  case "4k (3840 x 2160)":
    final_width=3840
    final_height=2160
  case "2 x original":
    final_width=2*video_width
    final_height=2*video_height
  case "3 x original":
    final_width=3*video_width
    final_height=3*video_height
  case "4 x original":
    final_width=4*video_width
    final_height=4*video_height

if aspect_ratio == 1.0 and "original" not in resolution:
  final_height = final_width

if aspect_ratio < 1.0 and "original" not in resolution:
  temp = final_width
  final_width = final_height
  final_height = temp

scale_factor = max(final_width/video_width, final_height/video_height)
isEven = int(video_width * scale_factor) % 2 == 0 and int(video_height * scale_factor) % 2 == 0

# scale_factor needs to be even
while isEven == False:
  scale_factor += 0.01
  isEven = int(video_width * scale_factor) % 2 == 0 and int(video_height * scale_factor) % 2 == 0

print(f"Upscaling from {video_width}x{video_height} to {final_width}x{final_height}, scale_factor={scale_factor}")

!python inference_realesrgan_video.py -n {model} -i '{video_path}' -o '{output_dir}' --outscale {scale_factor}

video_name_with_ext = os.path.basename(video_path)
video_name = video_name_with_ext.replace(".mp4","")
upscaled_video_path = f"{output_dir}{video_name}_out.mp4"
final_video_name = f"{video_name}_upscaled_{final_width}_{final_height}.mp4"
final_video_path = os.path.join(output_dir, final_video_name)

# crop to fit
if "original" not in resolution:
  print("Cropping to fit...")
  command = f"ffmpeg -loglevel error -hwaccel cuda -y -i '{upscaled_video_path}' -c:v h264_nvenc -filter:v  'crop={final_width}:{final_height}:(in_w-{final_width})/2:(in_h-{final_height})/2' -c:v libx264 -pix_fmt yuv420p '{final_video_path}'"
  subprocess.run(command,shell=True)

print(f"Upscaled video saved to: {final_video_path}")

# save to drive
if mount_drive:
  drive_folder = "MyDrive/Upscaled Videos (REAL-ESRGAN)"
  save_directory_drive = f"/content/gdrive/{drive_folder}"
  os.makedirs(save_directory_drive, exist_ok=True)

  command = f"'{final_video_path}' '{save_directory_drive}'/'{final_video_name}'"
  subprocess.run(command,shell=True)
  print(f"Saved to drive: /{drive_folder}/{final_video_name}" )

!rm "{upscaled_video_path}"

Upscaling from 1920x1080 to 3840x2160, scale_factor=2.0

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/content/Real-ESRGAN/inference_realesrgan_video.py", line 10, in <module>
    from basicsr.archs.rrdbnet_arch import RRDBNet
  File "/usr/local/lib/python3.11/dist-packages/basicsr/__init__.py", line 3, in <module>
    from .archs import *
  File "/usr/local/lib/python3.11/dist-packages/basicsr/archs/__init__.py", line 5, in <module>
    from basicsr.utils import get_root_logger, scandir
  File "/usr/local/lib/python3.11/dist-packages/basicsr/utils/__init__.py

# 3. Disconnect runtime

In [3]:
from google.colab import runtime

disconnect_when_finish = True  #@param{type:"boolean"}

if disconnect_when_finish:
  runtime.unassign()